# Observability

This notebook shows off different levels of LangChain observability. For more information, see:

- [LangChain Debugging Guide](https://python.langchain.com/docs/guides/debugging)

- [LangSmith](https://smith.langchain.com/)

## No Logging

In [13]:
import os
from dotenv import load_dotenv
from pathlib import Path
import openai

dotenv_path = Path('./.env')
load_dotenv(dotenv_path=dotenv_path)  # add your GOOGLE API key here

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
openai.api_key = os.getenv("OPEN_AI_KEY")

In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
search = TavilySearchResults()
tools = [search]
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [2]:
agent_executor.invoke({"input": "what is the weather in sf?"})

{'input': 'what is the weather in sf?',
 'output': "I found some information about the weather in San Francisco. According to the search results, the weather in San Francisco in January 2024 is expected to have an average temperature ranging from 52°F to 58°F. However, I couldn't find the current weather conditions. It's always a good idea to check a reliable weather website or app for the most up-to-date information."}

## `verbose=True`

In [3]:
from langchain.globals import set_verbose

set_verbose(True)

prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
search = TavilySearchResults()
tools = [search]
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

/Users/harrisonchase/.pyenv/versions/3.11.1/envs/langchain-0.1-guides/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:160: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [8]:
agent_executor.invoke({"input": "what is the weather in sf?"})

{'input': 'what is the weather in sf?',
 'output': "I'm sorry, but I couldn't find the current weather in San Francisco."}

## `debug=True`

In [10]:
from langchain.globals import set_debug

# set_verbose(True)
set_debug(True)

prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
search = TavilySearchResults()
tools = [search]
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [11]:
agent_executor.invoke({"input": "what is the weather in sf?"})

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "what is the weather in sf?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "what is the weather in sf?",
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": "what is the weather in sf?",
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": "what is the weather in sf?",
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad> > 5:chain:RunnableLambda] Entering Chain run with input:
{
  "input": "what is the weather in sf?",


{'input': 'what is the weather in sf?',
 'output': 'The weather in San Francisco is not available at the moment.'}

## LangSmith

In [14]:
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [15]:
set_debug(False)

agent_executor.invoke({"input": "what is the weather in sf?"})

{'input': 'what is the weather in sf?',
 'output': "I'm sorry, but I couldn't find the current weather in San Francisco. However, you can check the weather forecast for San Francisco on a reliable weather website or app."}